In [1]:
from train_eval_scripts_v2 import run_train, run_eval

## Liguistic importance

In [ ]:
device = 'cuda:0'

max_encoder_length = 256
source_cols_lists = [
    (["history#title", "history", "title"], 'response'), #NO grounding
    (["history#title#grounding", "history", "title", "grounding"], 'response'), #NO LINGUSTIC
    #(["history_aug_disco#title#grounding", "history_aug_disco", "title", "grounding", 'history_discourse'], 'response'), # +Discourse
    (["history_aug_disco#title#grounding", "history_aug_disco", "title", "grounding", 'history_discourse', 'history_amr', 'addr_amr'], 'response') # +AMR
] 

for source_cols, target_col in source_cols_lists:
    model_name_or_path = run_train(source_cols=source_cols,
                  target_col=target_col, #"response_aug",
                  max_encoder_length=max_encoder_length,
                  n_epochs=4, learning_rate=3e-5, batch_size=4, gradient_accumulation_steps=2, device=device)

    #run_eval(model_name_or_path, device=device)

In [ ]:
device = 'cuda:0'

max_encoder_length = 256
source_cols_lists = [
    (["history_aug#title#grounding", "history_aug", "title", "grounding", 'history_discourse', 'history_amr', 'addr_amr'], 'response'), # +sentiment
    (["history_aug#title#grounding", "history_aug", "title", "grounding", 'history_discourse', 'history_amr', 'addr_amr'], 'response_disco'), # +planning disco
    (["history_aug#title#grounding", "history_aug", "title", "grounding", 'history_discourse', 'history_amr', 'addr_amr'], 'response_aug') # +planning full
] 

for source_cols, target_col in source_cols_lists:
    model_name_or_path = run_train(source_cols=source_cols,
                  target_col=target_col, #"response_aug",
                  max_encoder_length=max_encoder_length,
                  n_epochs=4, learning_rate=3e-5, batch_size=4, gradient_accumulation_steps=2, device=device)

    #run_eval(model_name_or_path, device=device)

## Planning importance

In [2]:
device = 'cuda:0'

max_encoder_length = 256

In [ ]:
#no planning
model_name_or_path = run_train(source_cols=["history_aug#title#grounding", "grounding",
                                            'history_amr', 'addr_amr', 'history_discourse'],
      target_col="response",
      max_encoder_length=max_encoder_length,
      n_epochs=7, learning_rate=3e-5, batch_size=4, gradient_accumulation_steps=2, device=device)

In [3]:
#no lingustic at all
model_name_or_path = run_train(source_cols=["history#title#grounding", "title", "grounding"],
      target_col="response",
      max_encoder_length=max_encoder_length,
      n_epochs=7, learning_rate=3e-5, batch_size=4, gradient_accumulation_steps=2, device=device)

## Inference cycle

In [1]:
import os
import pandas as pd
import pickle
import re
import string
import subprocess
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np

import sys
sys.path.insert(0, 'multienc_bart_scripts_v2/')
sys.modules['transformers.generation_utils'] = __import__('generation_utils_custom')

from modeling_custom_bart import BartForConditionalGeneration
from transformers import BartTokenizer

In [2]:
source_lens = {
    'history': 600,
    'history_aug': 600,
    'history_aug_disco': 600,
    'history_amr': 1024,
    'history_discourse': 40,
    'addr_amr': 300,
    'response': 160,
    'response_aug': 160,
    'response_disco': 160,
    'grounding': 850,
    'title': 64,
    "history_aug#title#grounding": 1024,
    "history_aug_disco#title#grounding": 1024,
    "history#title#grounding": 1024,
    "history#title#grounding": 1024,
    "history#title": 600
}

In [3]:
def generate_top(input_texts, num_beams=4,  max_source_lens=[300, 200, 100], max_target_length=64,
                 top_k=50, top_p=1, temperature=1., do_sample=False):
    model_inputs_list = [tokenizer([input_texts[j]], max_length=max_source_lens[j],
                          padding="max_length" , truncation=True) for j in range(len(input_texts))]

    model_inputs = {}
    for key in model_inputs_list[0]:
        model_inputs[key] = [sum(el, []) for el in zip(*[model_inputs_list[j][key] for j in range(len(model_inputs_list))])]

    input_ids = torch.LongTensor(model_inputs['input_ids']).to(device)
    
    summary_ids = model.generate(input_ids, do_sample=do_sample, num_beams=num_beams,
                                 max_length=max_target_length, top_k=top_k, top_p=top_p, temperature=temperature)
    pred = tokenizer.batch_decode(summary_ids, clean_up_tokenization_spaces=False)[0]
    pred = re.sub(r'\s+', ' ', pred).replace('</s>', '').replace('<s>', '').strip()
    return pred

In [4]:
device = 'cpu'#'cuda:0'

In [5]:
chpath = 'multiencoder_bart_v2_bs8_4ep_lr3e-05_enclen256__from:history_aug#title#grounding-grounding-history_amr-addr_amr-history_discourse___to:response'

In [ ]:
for path in os.listdir('checkpoint'):
#for path in ['multiencoder_bart_v2_bs8_7ep_lr3e-05_enclen1__from:history_aug#title#grounding-title-grounding___to:response_aug']:
#for path in [chpath]:
    if '_v2_' in path:
        model_name_or_path = f'checkpoint/{path}'
        save_name = model_name_or_path.replace('checkpoint/', 'predictions/').replace('/checkpoint', '_') + 'short.pkl'
        if save_name.replace('predictions/', '') not in os.listdir('predictions'):
            try:
                tokenizer = BartTokenizer.from_pretrained(model_name_or_path)
                model = BartForConditionalGeneration.from_pretrained(model_name_or_path).train(False)

                model.to(device)

                source_cols = model_name_or_path.split('/')[1].split('__')[1][5:].split('-')
                target_col = model_name_or_path.split('/')[1].split('__')[-1][4:]
                max_encoder_length = int(re.findall(r'enclen(\d+)\D', model_name_or_path)[0])

                max_source_lengths = [source_lens[c] for c in source_cols]
                max_target_length = source_lens[target_col]

                #test_data = pd.read_csv("bart_input/val_reddit_dial_df_multi_extented.csv", sep='\t')
                #test_data = pd.read_csv("bart_input/val_reddit_dial_df_multi_extented_filt.csv", sep='\t')
                test_data = pd.read_csv("bart_input/val_reddit_dial_df_multi_extented_short.csv", sep='\t')
                preds = []

                for idx in tqdm(range(len(test_data))):
                    input_texts = list(test_data[source_cols].values[idx])
                    try:
                        pred = generate_top(input_texts,
                                            num_beams=1,
                                            max_source_lens=max_source_lengths,
                                            max_target_length=max_target_length,
                                            do_sample=True)
                    except:
                        pred = ""

                    preds.append(pred)

                with open(save_name, 'wb') as f:
                    pickle.dump(preds, f)

            except:
                continue

CUSTOM BART


100%|██████████| 15716/15716 [1:53:55<00:00,  2.30it/s]  


CUSTOM BART


100%|██████████| 15716/15716 [2:12:55<00:00,  1.97it/s]  


CUSTOM BART


100%|██████████| 15716/15716 [2:08:49<00:00,  2.03it/s]  


CUSTOM BART


 39%|███▉      | 6100/15716 [50:12<1:26:50,  1.85it/s]

### Single Generations

In [17]:
chpath = 'multiencoder_bart_v2_bs8_4ep_lr3e-05_enclen256__from:history_aug#title#grounding-history_aug-title-grounding-history_discourse-history_amr-addr_amr___to:response'

In [27]:
chpath = 'multiencoder_bart_v2_bs8_4ep_lr3e-05_enclen256__from:history_aug_disco#title#grounding-history_aug_disco-title-grounding-history_discourse___to:response'

In [28]:
model_name_or_path = f'checkpoint/{chpath}'
tokenizer = BartTokenizer.from_pretrained(model_name_or_path)
model = BartForConditionalGeneration.from_pretrained(model_name_or_path).train(False)

model.to(device)

source_cols = model_name_or_path.split('/')[1].split('__')[1][5:].split('-')
target_col = model_name_or_path.split('/')[1].split('__')[-1][4:]
max_encoder_length = int(re.findall(r'enclen(\d+)\D', model_name_or_path)[0])

max_source_lengths = [source_lens[c] for c in source_cols]
max_target_length = source_lens[target_col]

CUSTOM BART


In [51]:
def generate_custom(input_texts, num_beams=1,
                 top_k=50, top_p=1, temperature=1., do_sample=True):
    model_inputs_list = [tokenizer([input_texts[j]], max_length=max_source_lengths[j],
                          padding="max_length" , truncation=True) for j in range(len(input_texts))]

    model_inputs = {}
    for key in model_inputs_list[0]:
        model_inputs[key] = [sum(el, []) for el in zip(*[model_inputs_list[j][key] for j in range(len(model_inputs_list))])]

    input_ids = torch.LongTensor(model_inputs['input_ids']).to(device)
    
    summary_ids = model.generate(input_ids, do_sample=do_sample, num_beams=num_beams,
                                 max_length=max_target_length, top_k=top_k, top_p=top_p, temperature=temperature)
    pred = tokenizer.batch_decode(summary_ids, clean_up_tokenization_spaces=False)[0]
    pred = re.sub(r'\s+', ' ', pred).replace('</s>', '').replace('<s>', '').strip()
    return pred

In [67]:
test_data = pd.read_csv("bart_input/val_reddit_dial_df_multi_extented_filt.csv", sep='\t')

In [68]:
test_data.head()

,thread_id,id,history_aug,history_amr,history_discourse,addr_amr,response,response_aug,grounding,title,history,response_disco,history_aug_disco,history_aug_disco#title#grounding,history#title#grounding,history_aug#title#grounding,history#title
0,t3_zq3sz6,t3_zq3sz6_j0xvpbo,<s1> <u1> <to:u1> <init> <Negative> This is an...,( exemplify :ARG0 ( this ) :ARG1 ( disconnect ...,<u1> <to:u1> <init> </s> <u2> <to:u1> <elabora...,( multi-sentence :snt1 ( be-located-at :ARG1 (...,Yes and lets see if it fails and he leaves aft...,<agreement> <Negative> Yes and lets see if it ...,unk,James Gunn Confronts ‘Uproarious’ DC Backlash:...,<s1> <u1> <to:u1> This is an example of the di...,<agreement> Yes and lets see if it fails and h...,<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> This is an example of the di...,<s1> <u1> <to:u1> <init> <Negative> This is an...,<s1> <u1> <to:u1> This is an example of the di...
1,t3_zq3sz6,t3_zq3sz6_j0z6r1y,<s1> <u1> <to:u1> <init> <Negative> This is an...,( exemplify :ARG0 ( this ) :ARG1 ( disconnect ...,<u1> <to:u1> <init> </s> <u2> <to:u1> <elabora...,( cause :ARG0 ( have-degree :ARG1 ( build ) :A...,But if he isn't recasting everyone... I don't ...,<disagreement> <Neutral> But if he isn't recas...,unk,James Gunn Confronts ‘Uproarious’ DC Backlash:...,<s1> <u1> <to:u1> This is an example of the di...,<disagreement> But if he isn't recasting every...,<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> This is an example of the di...,<s1> <u1> <to:u1> <init> <Negative> This is an...,<s1> <u1> <to:u1> This is an example of the di...
2,t3_zq3sz6,t3_zq3sz6_j0xh7qu,<s1> <u1> <to:u1> <init> <Negative> This is an...,( exemplify :ARG0 ( this ) :ARG1 ( disconnect ...,<u1> <to:u1> <init> </s> <u2> <to:u1> <agreement>,( exact ) ( seem :ARG1 ( label :ARG1 ( or ) :A...,"Yeah, it makes sense. Like the above comment s...","<agreement> <Negative> Yeah, it makes sense. L...",unk,James Gunn Confronts ‘Uproarious’ DC Backlash:...,<s1> <u1> <to:u1> This is an example of the di...,"<agreement> Yeah, it makes sense. Like the abo...",<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> This is an example of the di...,<s1> <u1> <to:u1> <init> <Negative> This is an...,<s1> <u1> <to:u1> This is an example of the di...
3,t3_zq3sz6,t3_zq3sz6_j0x62se,<s1> <u1> <to:u1> <init> <Negative> This is an...,( exemplify :ARG0 ( this ) :ARG1 ( disconnect ...,<u1> <to:u1> <init> </s> <u2> <to:u1> <agreement>,( exact ) ( seem :ARG1 ( label :ARG1 ( or ) :A...,"I liked the Snyder films, Justice League might...","<unk> <Positive> I liked the Snyder films, Jus...",unk,James Gunn Confronts ‘Uproarious’ DC Backlash:...,<s1> <u1> <to:u1> This is an example of the di...,"<unk> I liked the Snyder films, Justice League...",<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> <init> This is an example of...,<s1> <u1> <to:u1> This is an example of the di...,<s1> <u1> <to:u1> <init> <Negative> This is an...,<s1> <u1> <to:u1> This is an example of the di...
4,t3_zq3sz6,t3_zq3sz6_j0x1xck,<s1> <u1> <to:u1> <init> <Negative> Not exactl...,( unite :polarity - :ARG1 ( fan :mod ( city ) ...,<u1> <to:u1> <init> </s> <u2> <to:u1> <disagre...,( contrast :ARG1 ( situation :mod ( win ) :mod...,Fuck 'em.I always said this of politics. The o...,<negativereaction> <Negative> Fuck 'em.I alway...,unk,James Gunn Confronts ‘Uproarious’ DC Backlash:...,<s1> <u1> <to:u1> Not exactly uniting the DC f...,<negativereaction> Fuck 'em.I always said this...,<s1> <u1> <to:u1> <init> Not exactly uniting t...,<s1> <u1> <to:u1> <init> Not exactly uniting t...,<s1> <u1> <to:u1> Not exactly uniting the DC f...,<s1> <u1> <to:u1> <init> <Negative> Not exactl...,<s1> <u1> <to:u1> Not exactly uniting the DC f...


In [69]:
input_texts = list(test_data[source_cols].values[10])

In [70]:
input_texts

['<s1> <u1> <to:u1> <init> I don’t think the next Batman reboot will be getting a solo movie for a loooooong time. After Reeves trilogy. </s> <s2> <u2> <to:u1> <answer> Many think that, but I don’t think it’s true. Gunn said that Batman is a major part of the reboot. I think, in order for the new DC Films to start forming, he’ll need to introduce the new Batman sooner rather than later. </s> <s3> <u3> <to:u2> <elaboration> Batman will be introduced, that doesn’t mean he needs to have his own Standalone films. </s> <s1> <u4> <to:u3> Batman will be introduced, that doesn’t mean he needs to have his own Standalone films. </s> Who should be the main villain of James Gunn\'s Batman? </s> The Batman is a 2022 American superhero film based on the DC Comics character Batman. Produced by DC Films, 6th & Idaho, and Dylan Clark Productions, and distributed by Warner Bros. Pictures, it is a reboot of the Batman film franchise. The film was directed by Matt Reeves, who wrote the screenplay with Pet

In [71]:
generate_custom(input_texts)

'You’re right that Batman was Superman in the first comic. There will be a Batman to be introduced for the DCU. The Justice League and the upcoming Justice League films will be Batman shows. The Batman and the Justice League events (Jokes for Justice League) are in the ongoing DCEU. There’s just no way a Batman story is going to have all that much exposition. Gunn is on his way to throwing in a Batman origin (maybe with Bruce Wayne and Matt Reeves, not Superman).'